In [1]:
import re
import time

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml

import dill

In [2]:
def get_thing(id):
    '''A "thing" is BGG's designation for a physical item, such as a board game,
       expansion, board game accessory, etc.  The "id" supplied can have several numbers
       separated by commas to retrieve more than one item at a time.
    
       For more information see: https://boardgamegeek.com/wiki/page/BGG_XML_API2#
    '''
    url = 'https://www.boardgamegeek.com/xmlapi2/thing?id=' + str(id).strip()
    r = requests.get(url)
    if r.status_code == 404:
        return None
    while r.status_code == 202:
        time.sleep(6)
        r = requests.get(url)
    return re.sub('[\n\t]', '', r.text)

def add_options(url, own=None, preordered=None, prevowned=None, fortrade=None, want=None, 
                wanttobuy=None, wanttoplay=None, wishlist=None):
    if own in [0,1]:
        url += '&own=' + str(own)
    if prevowned in [0,1]:
        url += '&prevowned=' + str(prevowned)
    if preordered in [0,1]:
        url += '&preordered=' + str(preordered)
    if fortrade in [0,1]:
        url += '&fortrade=' + str(fortrade)
    if want in [0,1]:
        url += '&want=' + str(want)
    if wishlist in [0,1]:
        url += '&wishlist=' + str(wishlist)
    if wanttobuy in [0,1]:
        url += '&wanttobuy=' + str(wanttobuy)
    if wanttoplay in [0,1]:
        url += '&wanttoplay=' + str(wanttoplay)
    return url

def get_collection(bgg_user_id, own=None, preordered=None, prevowned=None, fortrade=None, want=None, 
                   wanttobuy=None, wanttoplay=None, wishlist=None):
    '''For more information see:  https://boardgamegeek.com/wiki/page/BGG_XML_API2

       Get the board games, and then get the board game expansions.  This is a quirk of the 
       BGG xmlapi2 interface, in that it will incorrectly return the expansions as subtype="boardgame",
       so we make two calls to get the boardgames, and then the expansions separately.
    '''
    url = 'https://www.boardgamegeek.com/xmlapi2/collection?username=' + bgg_user_id.strip() + '&excludesubtype=boardgameexpansion&stats=1'
    #  Add parameters to the url based on what was passed to this function.
    url = add_options(url, own, preordered, prevowned, fortrade, want, wanttobuy, wanttoplay, wishlist)
    r = requests.get(url)
    if r.status_code == 404:
        return None
    while r.status_code == 202:   ##  BGG says that it usually queues requests for a collection, so we 
                                  ##  must check for a 202 code, and sleep and try again if necessary.  
        time.sleep(8)
        r = requests.get(url)
    first_result = re.sub('[\n\t]', '', r.text)
    first_result = list(BeautifulSoup(first_result, 'lxml').find_all('item'))

    url = 'https://www.boardgamegeek.com/xmlapi2/collection?username=' + bgg_user_id.strip() + '&subtype=boardgameexpansion&stats=1'
    #  Add parameters to the url based on what was passed to this function.
    url = add_options(url, own, preordered, prevowned, fortrade, want, wanttobuy, wanttoplay, wishlist)
    r = requests.get(url)
    while r.status_code == 202:   
        time.sleep(8)
        r = requests.get(url)
    second_result = re.sub('[\n\t]', '', r.text)
    second_result = list(BeautifulSoup(second_result, 'lxml').find_all('item'))
    
    if not second_result:
        pass
    first_result.extend(second_result)
    
    glist = []
    for item in first_result:
        d = dict()
        d['objectid'] = item.attrs['objectid']
        d['subtype'] = item.attrs['subtype']
        if item.find('yearpublished'):
            d['yearpublished'] = item.find('yearpublished').text
        d['name'] = item.find('name').text
        d.update(item.find("status").attrs)
        d['numplays'] = item.find('numplays').text
        d['lastmodified'] = pd.to_datetime(d['lastmodified'])
        if item.find('rating'):
            d['rating'] = item.find('rating').attrs['value']
        if item.find('comment'):
            d['comment'] = item.find('comment').text
        glist.append(d)
    
    glist = pd.DataFrame(glist, columns=['objectid','subtype','name','yearpublished','own','prevowned','fortrade','want','wanttoplay','wanttobuy','wishlist','preordered','lastmodified','numplays','rating','comment'])
    for column in ['objectid', 'yearpublished', 'own', 'prevowned', 'fortrade', 'want', 'wanttoplay', 'wanttobuy', 'wishlist', 'preordered', 'numplays']:
        glist[column].fillna(-1, inplace=True)
        glist[column] = glist[column].astype(np.int32)
    #glist['lastmodified'] = glist['lastmodified'].to_datetime()
    
    return glist

In [3]:
pd.to_datetime?

In [4]:
c = get_collection('craw-daddy', prevowned=0)

In [5]:
c

,objectid,subtype,name,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,numplays,rating,comment
0,155122,boardgame,"1066, Tears To Many Mothers",2018,0,0,0,0,0,0,0,1,2018-02-14 04:20:28,0,N/A,NaN
1,177590,boardgame,13 Days: The Cuban Missile Crisis,2015,1,0,0,0,0,0,0,0,2016-03-17 02:39:31,4,N/A,NaN
2,199269,boardgame,1572: The Lost Expedition,2016,0,0,0,0,1,0,0,0,2017-07-11 09:08:01,0,N/A,NaN
3,421,boardgame,1830: Railways & Robber Barons,2011,1,0,0,0,0,0,0,0,2016-03-13 16:03:15,2,N/A,NaN
4,183308,boardgame,1844/1854,2016,1,0,0,0,0,0,0,0,2016-05-03 12:20:40,1,N/A,NaN
5,17405,boardgame,1846: The Race for the Midwest,2016,1,0,0,0,0,0,0,0,2016-11-21 13:53:20,2,N/A,NaN
6,12750,boardgame,1860: Railways on the Isle of Wight,2010,1,0,0,0,0,0,0,0,2016-02-15 13:57:31,2,7,NaN
7,66837,boardgame,1862: Railway Mania in the Eastern Counties,2013,1,0,0,0,0,0,0,0,2015-01-12 01:52:19,0,N/A,NaN
8,23540,boardgame,1889: History of Shikoku Railways,2005,0,0,0,0,1,0,0,0,2017-09-16 17:43:22,0,N/A,NaN
9,2612,boardgame,18AL,2013,1,0,0,0,0,0,0,0,2016-10-22 08:02:01,2,N/A,NaN


In [6]:
c.dtypes

objectid                  int32
subtype                  object
name                     object
yearpublished             int32
own                       int32
prevowned                 int32
fortrade                  int32
want                      int32
wanttoplay                int32
wanttobuy                 int32
wishlist                  int32
preordered                int32
lastmodified     datetime64[ns]
numplays                  int32
rating                   object
comment                  object
dtype: object

In [7]:
c[c['lastmodified'].isna()]

,objectid,subtype,name,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,numplays,rating,comment


In [10]:
c[c['wishlist'] == 1]

,objectid,subtype,name,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,numplays,rating,comment
50,4616,boardgame,Arimaa,2002,0,0,0,0,1,0,1,0,2013-11-07 10:36:58,0,N/A,NaN
56,245456,boardgame,Attack of the 50 Foot Colossi,2018,0,0,0,0,0,0,1,0,2018-11-12 19:36:38,0,N/A,NaN
59,231581,boardgame,AuZtralia,2018,0,0,0,0,0,0,1,0,2018-11-12 19:32:08,0,N/A,NaN
133,197376,boardgame,Charterstone,2017,0,0,0,0,0,0,1,0,2018-11-12 17:40:47,0,N/A,NaN
169,172558,boardgame,Crashland,2015,0,0,0,0,1,0,1,0,2016-11-01 17:38:10,0,N/A,NaN
184,192802,boardgame,Days of Ire: Budapest 1956,2016,0,0,0,0,0,0,1,0,2016-11-01 17:26:30,0,N/A,NaN
215,156180,boardgame,Eggs and Empires,2014,0,0,0,0,1,0,1,0,2016-11-14 11:44:50,0,N/A,NaN
225,188390,boardgame,Enemy Coast Ahead: The Doolittle Raid,-1,0,0,0,0,1,0,1,0,2016-11-05 11:55:14,0,N/A,NaN
251,257733,boardgame,Fine Sand,2018,0,0,0,0,0,0,1,0,2018-10-27 10:46:19,0,N/A,NaN
260,176262,boardgame,Fleet Wharfside,2015,0,0,0,0,0,0,1,0,2015-11-07 16:04:33,0,N/A,NaN


In [8]:
c = get_collection('Hopalong')

In [9]:
c

,objectid,subtype,name,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,numplays,rating,comment
0,31795,boardgame,"1, 2 oder 3",2007,1,0,0,0,0,0,0,0,2007-09-14 12:05:54,0,N/A,Spielbox insert. Unplayed
1,7865,boardgame,10 Days in Africa,2003,0,0,0,0,0,0,1,0,2016-08-27 15:08:16,2,7,Fun filler with luck in the tile draw so a gam...
2,7866,boardgame,10 Days in the USA,2003,1,0,0,0,0,0,0,0,2017-03-26 16:51:38,2,7,"Fun filler, similar to 10 Days in Africa, but ..."
3,63706,boardgame,11 nimmt!,2010,1,0,0,0,0,0,0,0,2017-03-29 13:08:48,8,8,"Similar cards to 6 nimmt, but the multiple bul..."
4,170430,boardgame,12 Days of Christmas,2015,1,0,0,0,0,0,0,0,2017-01-25 11:43:24,0,N/A,NaN
5,7217,boardgame,1500 Gold,1995,0,0,0,0,0,0,1,0,2016-08-27 15:19:46,0,N/A,NaN
6,3312,boardgame,1776,1974,1,0,0,0,0,0,0,0,2016-08-27 15:21:47,0,N/A,Unplayed
7,63170,boardgame,1817,2010,1,0,0,0,0,0,0,0,2017-01-25 14:36:16,0,N/A,NaN
8,193867,boardgame,1822: The Railways of Great Britain,2016,0,0,0,0,0,0,1,0,2016-07-08 05:22:12,0,N/A,NaN
9,937,boardgame,1825 Unit 1,1995,1,0,0,0,0,0,0,0,2017-01-26 09:39:24,1,N/A,Was lucky to buy all 3 units together.


In [10]:
c[c['wishlist'] == 1]

,objectid,subtype,name,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,numplays,rating,comment
1,7865,boardgame,10 Days in Africa,2003,0,0,0,0,0,0,1,0,2016-08-27 15:08:16,2,7,Fun filler with luck in the tile draw so a gam...
5,7217,boardgame,1500 Gold,1995,0,0,0,0,0,0,1,0,2016-08-27 15:19:46,0,N/A,NaN
8,193867,boardgame,1822: The Railways of Great Britain,2016,0,0,0,0,0,0,1,0,2016-07-08 05:22:12,0,N/A,NaN
18,7935,boardgame,1844: Switzerland,2003,0,0,0,0,0,0,1,0,2017-10-01 13:38:39,0,N/A,Perhaps superceded by the 1844/54 game from Lo...
27,87907,boardgame,1865: Sardinia,2011,0,0,0,0,0,0,1,0,2016-08-27 16:01:17,3,8,Very different from other 18xx games in that c...
42,187104,boardgame,4 Gods,2016,0,0,0,0,0,0,1,0,2017-05-24 06:15:00,0,N/A,NaN
47,175878,boardgame,504,2015,0,0,0,0,0,0,1,0,2016-10-15 10:12:41,3,7,NaN
49,192458,boardgame,51st State: Master Set,2016,0,0,0,0,0,0,1,0,2016-09-23 10:38:40,0,N/A,NaN
57,161537,boardgame,7 Steps,2014,0,0,0,0,0,0,1,0,2015-10-06 04:35:06,0,N/A,NaN
72,177857,boardgame,Achaia,2015,0,0,0,0,0,0,1,0,2016-09-23 10:37:59,0,N/A,NaN


In [40]:
helixx = get_collection('Helixx')

In [42]:
helixx[helixx['wishlist'] == 1]

,objectid,subtype,name,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,numplays,rating,comment
3,193867,boardgame,1822: The Railways of Great Britain,2016,0,0,0,0,1,0,1,0,2017-11-17 17:02:21,0,N/A,NaN
9,38343,boardgame,Ad Astra,2009,0,0,0,0,0,0,1,0,2015-11-01 11:23:42,0,N/A,NaN
34,205346,boardgame,Beer Empire,2016,0,0,0,0,1,0,1,0,2016-11-01 10:34:39,0,N/A,NaN
53,172081,boardgame,Burgle Bros.,2015,0,0,0,0,0,0,1,0,2018-11-12 08:44:11,3,N/A,NaN
60,245934,boardgame,Carpe Diem,2018,0,0,0,0,0,0,1,0,2018-11-12 08:25:09,1,N/A,NaN
63,102794,boardgame,Caverna: The Cave Farmers,2013,0,0,0,0,0,0,1,0,2018-11-12 08:25:26,7,N/A,NaN
65,248900,boardgame,Ceylon,2018,0,0,0,0,0,0,1,0,2018-11-12 08:42:06,0,N/A,NaN
75,245638,boardgame,Coimbra,2018,0,0,0,0,0,0,1,0,2018-11-12 08:45:10,0,N/A,NaN
117,249381,boardgame,The Estates,2018,0,0,0,0,0,0,1,0,2018-11-12 11:38:23,1,N/A,NaN
121,183284,boardgame,Factory Funner,2016,0,0,0,0,0,0,1,0,2018-11-12 08:40:12,2,N/A,NaN


In [11]:
##  Retrieve all of the boardgame categories used by BGG for classification.

page = requests.get('https://boardgamegeek.com/advsearch/boardgame')
soup = BeautifulSoup(page.text,"lxml")
soup.get("boardgamecategory")
gc = soup.find("div",{"id":"boardgamecategory"})
catValues = gc.findAll("input",{"name":"propertyids[]"})
catValues = [int(x.attrs['value']) for x in catValues]
gc.findAll("td",{"width":"70%"})
cats = [x.text for x in gc.findAll("td",{"width":"70%"})]

l = list(zip(catValues, cats))

In [12]:
boardGameCategory = pd.DataFrame(l,columns=['id','category'])

In [13]:
boardGameCategory.dtypes

id           int64
category    object
dtype: object

In [34]:
boardGameCategory.set_index('id', inplace=True)

In [35]:
boardGameCategory

,category
id,
1009,Abstract Strategy
1032,Action / Dexterity
1022,Adventure
2726,Age of Reason
1048,American Civil War
1108,American Indian Wars
1075,American Revolutionary War
1055,American West
1050,Ancient


In [36]:
with open('data/boardGameCategory.dill','wb') as f:
    dill.dump(boardGameCategory, f)

In [16]:
#  Get the mechanic categories used by BGG for describing games' mechanisms of play.
gm = soup.find("div",{'id':'boardgamemechanic'})
gm.findAll('input',{'name':'propertyids[]'})
gm_values=[int(x.attrs['value']) for x in gm.findAll('input',{'name':'propertyids[]'})]
gm_strings=[x.text for x in gm.findAll('td',{'width':'70%'})]

l = list(zip(gm_values, gm_strings))

boardGameMechanic = pd.DataFrame(l, columns=['id','mechanic'])

In [37]:
boardGameMechanic.set_index('id', inplace=True)

In [38]:
boardGameMechanic

,mechanic
id,
2073,Acting
2689,Action / Movement Programming
2001,Action Point Allowance System
2080,Area Control / Area Influence
2043,Area Enclosure
2046,Area Movement
2021,Area-Impulse
2012,Auction/Bidding
2014,Betting/Wagering


In [39]:
with open('data/boardGameMechanic.dill','wb') as f:
    dill.dump(boardGameMechanic,f)

In [19]:
i = get_thing(1)

In [20]:
type(i)

str

In [21]:
url = 'https://www.boardgamegeek.com/xmlapi2/thing?id=1'
r = requests.get(url)

In [22]:
type(r)

requests.models.Response

In [23]:
result = BeautifulSoup(r.text, "lxml")

In [24]:
result

<?xml version="1.0" encoding="utf-8"?><html><body><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item id="1" type="boardgame">
<thumbnail>https://cf.geekdo-images.com/thumb/img/RgXAhOreEqPeNiPpDPEUTwLm5Wk=/fit-in/200x150/pic159509.jpg</thumbnail>
<image>https://cf.geekdo-images.com/original/img/vOttDcPBg1Tas9F6vFDhRmVaNH8=/0x0/pic159509.jpg</image>
<name sortindex="5" type="primary" value="Die Macher"></name>
<description>Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well your party does in it. Second, if a party wins a regional election and has some m

In [25]:
result.find("item").attrs['id']

'1'

In [26]:
result.find("name").attrs['value']

'Die Macher'

In [27]:
result.find("description").text

'Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well your party does in it. Second, if a party wins a regional election and has some media influence in the region, then the party will receive some media-control victory points. Third, each party has a national party membership which will grow as the game progresses and this will supply a fair number of victory points. Lastly, parties score some victory points if their party platform matches the national opinions at the end of the game.&#10;&#10;The 1986 edition featured four parties from the old West Germany and supported

In [28]:
result.find("yearpublished").attrs['value']

'1986'

In [29]:
result.find("minplayers").attrs['value']

'3'

In [30]:
result.find("maxplayers").attrs['value']

'5'

In [31]:
#  Board game categories  
catDict = {link.attrs['value'] : 1 for link in result.findAll("link", {"type":"boardgamecategory"})}
catDict

{'Economic': 1, 'Negotiation': 1, 'Political': 1}

In [32]:
#  Mechanics in the game
mechDict = {link.attrs['value'] : 1 for link in result.findAll('link', {'type':'boardgamemechanic'})}
mechDict

{'Area Control / Area Influence': 1,
 'Auction/Bidding': 1,
 'Dice Rolling': 1,
 'Hand Management': 1,
 'Simultaneous Action Selection': 1}

In [33]:
#  "Families" of the game
familyDict = {link.attrs['value'] : 1 for link in result.findAll('link', {'type':'boardgamefamily'})}
familyDict

{'Country: Germany': 1,
 'Political: Elections': 1,
 'Valley Games Classic Line': 1}

In [43]:
int(True)

1